### Import das dependencias

In [25]:
import os

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### 1. Importar todos os datasets tratados

In [26]:
data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

consumos_faturados_path = data_path + 'treated/consumos_faturados_por_municipio_10_anos.csv'
caracterizacao_cpe_path = data_path + 'treated/caracterizacao-pes-contrato-ativo.csv'
diagrama_carga_path = data_path + 'treated/diagrama_de_carga_por_mes.csv'

df_consumos_faturados = pd.read_csv(consumos_faturados_path)
df_caracterizacao_cpe = pd.read_csv(caracterizacao_cpe_path)
df_diagrama_carga = pd.read_csv(diagrama_carga_path)

### 2. Juntar os datasets

- 1. Primeiro juntar consumos faturados e caracterizacao cpe
    - A coleção de caracterizacao CPE tem uma categorização de nível de tensão diferente da coleção consumos faturados
        - Separando a "Baixa Tensão" em duas "Baixa Tensão Normal" e "Baixa Tensão Especial"
        - Temos então de tratar da coleção caracterizacao_cpe primeiro para tornar esses dois tipos num único tipo "Baixa Tensão"
    - Após esse tratamento, pode-se proceder à junção 
- 2. Segundo juntar com a coleção de diagramas de carga.

Ver os valores possíveis de Nível de Tensão da coleção caracterizacao de cpe

In [27]:
df_caracterizacao_cpe['Nível de Tensão'].unique()

array(['Baixa Tensão Especial', 'Baixa Tensão Normal',
       'Muito Alta, Alta e Média Tensões'], dtype=object)

1. Tratamento coleção caracterizacao CPE

In [28]:
# Passo 1: Substituir valores de nível de tensão
df_caracterizacao_cpe['Nível de Tensão'] = df_caracterizacao_cpe['Nível de Tensão'].replace({'Baixa Tensão Normal': 'Baixa Tensão', 'Baixa Tensão Especial': 'Baixa Tensão'})

df_caracterizacao_cpe['Nível de Tensão'].unique()

array(['Baixa Tensão', 'Muito Alta, Alta e Média Tensões'], dtype=object)

2. Agrupar os valores de contagens de CPEs com a nova categoria "Baixa Tensão"

In [29]:
df_caracterizacao_cpe_grouped = df_caracterizacao_cpe.groupby(['Concelho', 'Nível de Tensão', 'Tipo de Instalação', 'Ano', 'Mês']).sum().reset_index()

df_caracterizacao_cpe_grouped


,Concelho,Nível de Tensão,Tipo de Instalação,Ano,Mês,CPEs (#)
0,Abrantes,Baixa Tensão,Doméstico,2022,3,20283
1,Abrantes,Baixa Tensão,Doméstico,2022,4,20267
2,Abrantes,Baixa Tensão,Doméstico,2022,5,20254
3,Abrantes,Baixa Tensão,Doméstico,2022,6,20259
4,Abrantes,Baixa Tensão,Doméstico,2022,7,20250
...,...,...,...,...,...,...
35123,Óbidos,"Muito Alta, Alta e Média Tensões",Não Doméstico,2023,11,66
35124,Óbidos,"Muito Alta, Alta e Média Tensões",Não Doméstico,2023,12,66
35125,Óbidos,"Muito Alta, Alta e Média Tensões",Não Doméstico,2024,1,66
35126,Óbidos,"Muito Alta, Alta e Média Tensões",Não Doméstico,2024,2,66


3. Guardar ficheiro para verificar se está tudo certo

In [30]:
file_path = data_path + 'treated/caracterizacao-pes-contrato-ativo-agrupado.csv'

df_caracterizacao_cpe_grouped.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)

DataFrame guardado em  ../data/treated/caracterizacao-pes-contrato-ativo-agrupado.csv


4. Junção das coleçoes consumos faturados e caracterizacao cpe

In [31]:
merged_df = pd.merge(df_consumos_faturados, df_caracterizacao_cpe_grouped, on=['Concelho', 'Ano', 'Mês', 'Nível de Tensão'], how='inner')

# Verificar nulls
null_counts = merged_df.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

merged_df.info()
merged_df.head(50)

Número de nulls por coluna:
Concelho               0
Nível de Tensão        0
Ano                    0
Mês                    0
Energia Ativa (kWh)    0
Tipo de Instalação     0
CPEs (#)               0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32278 entries, 0 to 32277
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Concelho             32278 non-null  object 
 1   Nível de Tensão      32278 non-null  object 
 2   Ano                  32278 non-null  int64  
 3   Mês                  32278 non-null  int64  
 4   Energia Ativa (kWh)  32278 non-null  float64
 5   Tipo de Instalação   32278 non-null  object 
 6   CPEs (#)             32278 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 1.7+ MB


,Concelho,Nível de Tensão,Ano,Mês,Energia Ativa (kWh),Tipo de Instalação,CPEs (#)
0,Abrantes,Baixa Tensão,2022,3,6375824.308,Doméstico,20283
1,Abrantes,Baixa Tensão,2022,3,6375824.308,Iluminação Pública,293
2,Abrantes,Baixa Tensão,2022,3,6375824.308,Não Doméstico,4334
3,Abrantes,Baixa Tensão,2022,3,6375824.308,Outros,83
4,Abrantes,Baixa Tensão,2022,4,5560992.246,Doméstico,20267
5,Abrantes,Baixa Tensão,2022,4,5560992.246,Iluminação Pública,293
6,Abrantes,Baixa Tensão,2022,4,5560992.246,Não Doméstico,4337
7,Abrantes,Baixa Tensão,2022,4,5560992.246,Outros,83
8,Abrantes,Baixa Tensão,2022,5,5480833.643,Doméstico,20254
9,Abrantes,Baixa Tensão,2022,5,5480833.643,Iluminação Pública,293


5. Juntar com a coleção diagramas de carga

In [32]:
merged_df = pd.merge(merged_df, df_diagrama_carga, on=['Concelho', 'Ano', 'Mês'], how='inner')

# Verificar nulls
null_counts = merged_df.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

merged_df.info()
merged_df.head(50)

Número de nulls por coluna:
Concelho                       0
Nível de Tensão                0
Ano                            0
Mês                            0
Energia Ativa (kWh)            0
Tipo de Instalação             0
CPEs (#)                       0
Inclui Mobilidade Elétrica     0
CPEs com DCs Recolhidos (#)    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49195 entries, 0 to 49194
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Concelho                     49195 non-null  object 
 1   Nível de Tensão              49195 non-null  object 
 2   Ano                          49195 non-null  int64  
 3   Mês                          49195 non-null  int64  
 4   Energia Ativa (kWh)          49195 non-null  float64
 5   Tipo de Instalação           49195 non-null  object 
 6   CPEs (#)                     49195 non-null  int64  
 7   Inclui Mobilidade Elétrica 

,Concelho,Nível de Tensão,Ano,Mês,Energia Ativa (kWh),Tipo de Instalação,CPEs (#),Inclui Mobilidade Elétrica,CPEs com DCs Recolhidos (#)
0,Abrantes,Baixa Tensão,2022,8,5906036.429,Doméstico,20260,Não,11097
1,Abrantes,Baixa Tensão,2022,8,5906036.429,Doméstico,20260,Sim,1
2,Abrantes,Baixa Tensão,2022,8,5906036.429,Iluminação Pública,293,Não,11097
3,Abrantes,Baixa Tensão,2022,8,5906036.429,Iluminação Pública,293,Sim,1
4,Abrantes,Baixa Tensão,2022,8,5906036.429,Não Doméstico,4318,Não,11097
5,Abrantes,Baixa Tensão,2022,8,5906036.429,Não Doméstico,4318,Sim,1
6,Abrantes,Baixa Tensão,2022,8,5906036.429,Outros,83,Não,11097
7,Abrantes,Baixa Tensão,2022,8,5906036.429,Outros,83,Sim,1
8,Abrantes,Baixa Tensão,2022,9,5316878.736,Doméstico,20270,Não,11298
9,Abrantes,Baixa Tensão,2022,9,5316878.736,Doméstico,20270,Sim,1


6. Guardar ficheiro para verificar se está tudo certo

In [33]:
file_path = data_path + 'treated/merged-dataset.csv'

merged_df.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)

DataFrame guardado em  ../data/treated/merged-dataset.csv
